In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.42B.300d'
config = {
    'max_size': 64,
    'min_freq': 5,
    'batch_size': 64
}


In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [4]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['Sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'Harvey', 'Nichols', "'", 'hilarious', 'Christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '3.333333333333333'} {'Headline': ['Three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [6]:
max_len = 0
for i in range(i):
    if len(AttractiveData.test_data[i].Headline) >= max_len:
        max_len = len(AttractiveData.test_data[i].Headline)
max_len

31

## Start to train

In [7]:
num_workers = 10

config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 16
config['hidden_dim'] = 256
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 200
config['lr'] = {
    'encoder': 1e-4,
    'embedding': 1e-5,
    'linear': 1e-4
}
config['num_layers'] = 3
config['nhead'] = 4
config['dropout'] = 0.1

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1519, 300])


In [8]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [9]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, pretrained_embeddings)

In [10]:
AttractiveTrainer.model

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1519, 300, padding_idx=1)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
  (encoder): LSTM(300, 256, num_layers=3, dropout=0.1)
  (linear): Linear(in_features=272, out_features=1, bias=True)
)

In [11]:
AttractiveTrainer.train()

ch 80
EP: train | lr: {'encoder': 0.0001, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 195.64it/s]

EP_train | avg_loss: 0.7981985304504633 |
Epoch 81
EP: train | lr: {'encoder': 0.0001, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 194.27it/s]

EP_train | avg_loss: 0.7907169256359339 |
Epoch 82
EP: train | lr: {'encoder': 0.0001, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 195.36it/s]

EP_train | avg_loss: 0.7857140302658081 |
Epoch 83
EP: train | lr: {'encoder': 0.0001, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 194.89it/s]

EP_train | avg_loss: 0.7745080664753914 |
Epoch 84
EP: train | lr: {'encoder': 0.0001, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 193.60it/s]

EP_train | avg_loss: 0.7726117614656687 |
Epoch 85
EP: train | lr: {'encoder': 0.0001, 'embedding': 1e-05, 'linear': 0.0001}: 100%|| 32/32 [00:00<00:00, 192.59it/s]

EP_train | avg_loss: 0.7638187948614359 |
Epoch 8

## for classification, not better

In [14]:
# from sklearn.metrics import mean_squared_error
# a = AttractiveTrainer.train_predict
# AttractiveData.LABEL.vocab.itos[int(a[0])], AttractiveTrainer.train_true[0]
# correct = 0
# pred_list = []
# true_list = []
# for i in range(len(a)):
#     pred = AttractiveData.LABEL.vocab.itos[int(a[i])]
#     pred_list.append(float(pred))
#     true = AttractiveData.LABEL.vocab.itos[int(AttractiveTrainer.train_true[i])]
#     true_list.append(float(true))
# mean_squared_error(true_list, pred_list)
# # true_list

0.5601443355119825

## Below is testing

In [13]:
from transformermodel import TransformerModel
from attractivenet import AttractiveNet
PATH = './model/LSTM_20201030-162411_0.604.200'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1519, 300, padding_idx=1)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(18, 16, padding_idx=0)
  (encoder): LSTM(300, 256, num_layers=3, dropout=0.1)
  (linear): Linear(in_features=272, out_features=1, bias=True)
)

In [14]:
def predict_attractive(sentence, category):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)
    tensor_category = tensor_category

    prediction = load_model(tensor_sentence, tensor_category)
    
    return prediction

In [18]:
# train mean = 3.2, test mean = 2.8
predict_list = []
for i, sentence in enumerate(AttractiveData.test_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    # predict_list.append(prediction.item() - 3.2 + 2.8)
    predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

In [19]:
train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
for each_test in test_category:
    if each_test not in train_category:
        print(each_test)
print()
for each_train in train_category:
    if each_train not in test_category:
        print(each_train)


golf
beauty


In [17]:
train_category, test_category

(['travel',
  'health',
  'femail',
  'sport',
  'gardening',
  'sciencetech',
  'news',
  'food',
  'football',
  'travelnews',
  'cricket',
  'golf',
  'books',
  'rugbyunion',
  'home',
  'boxing',
  'tennis',
  'concussion',
  'othersports',
  'beauty',
  'formulaone',
  'racing'],
 ['health',
  'femail',
  'sciencetech',
  'travel',
  'news',
  'football',
  'food',
  'living',
  'books',
  'boxing',
  'rugbyunion',
  'othersports',
  'formulaone',
  'cricket',
  'us',
  'tennis',
  'sport',
  'middleeast',
  'racing'])

## Below just for fun guess

In [17]:
a = AttractiveData.df_test['ID'].to_list()

In [13]:
from sklearn.metrics import mean_squared_error

In [9]:
mean_squared_error(a, b)
# Training all 3.0 got mse = 0.5545

0.5545343137254902

In [18]:
# only for fun
import random
guess_list = [2.3333333333333335, 3.3333333333333335, 3.6666666666666665, 2.6666666666666665]
b = []
for i in range(len(a)):
    b.append(random.choice(guess_list))

In [19]:
AttractiveData.df_test['Label'] = b

In [20]:
AttractiveData.df_test[['ID', 'Label']].to_csv('all_3.csv', index=False)